# Browsing Agent
This notebook will use the internet to download content like pdfs. Then an LLM will decide which files to keep. Next it will create a vector index of the files, for easy RAG.

In [4]:
#imports
%load_ext autoreload
%autoreload 2
import os
from open_agent import OpenAgent
from config import Config
from IPython.display import display
import numpy as np
from transformers import CLIPProcessor, CLIPModel
import requests
from googlesearch import search
import PyPDF2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Function to search for PDF URLs using a query
def search_pdf_urls(query, num_results=10):
    pdf_urls = []
    for url in search(query, num_results=num_results):
        # Optionally check if the URL really ends with '.pdf'
        if url.lower().endswith(".pdf"):
            pdf_urls.append(url)
    return pdf_urls

In [6]:
# Function to download a PDF file from a given URL
def download_pdf(url, dest_folder="downloads"):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    local_filename = os.path.join(dest_folder, url.split("/")[-1])
    try:
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(local_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"Downloaded: {local_filename}")
        return local_filename
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

In [7]:
# Function to extract metadata (e.g., creation date) from the PDF
def extract_pdf_metadata(pdf_path):
    metadata = {}
    try:
        with open(pdf_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            metadata = reader.metadata
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    return metadata

In [8]:
# Function to extract text from the first few pages of the PDF
def extract_pdf_text(pdf_path, max_pages=3):
    text = ""
    try:
        with open(pdf_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            num_pages = len(reader.pages)
            pages_to_read = min(num_pages, max_pages)
            for i in range(pages_to_read):
                page = reader.pages[i]
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
    return text

In [11]:
brand = "mölnycke mepilex border flex"
product_type = "product sheet"

# Construct the search query. The filetype operator helps to target PDFs.
query = f"{brand} {product_type} filetype:pdf"
print("Searching for PDFs...")
pdf_urls = search_pdf_urls(query, num_results=10)
print(f"Found {len(pdf_urls)} PDF URLs.")

# Process each found PDF
for url in pdf_urls:
    print(f"\nProcessing URL: {url}")
    file_path = download_pdf(url)
    if not file_path:
        continue
    
    # Extract metadata such as creation date
    metadata = extract_pdf_metadata(file_path)
    creation_date = metadata.get("/CreationDate", "Unknown")
    print(f"Creation Date (from metadata): {creation_date}")
    
    # Extract text from the PDF (first few pages)
    pdf_text = extract_pdf_text(file_path)

Searching for PDFs...
Found 7 PDF URLs.

Processing URL: https://www.molnlycke.com/globalassets/mepilex-border-flex-hqim005406.pdf
Downloaded: downloads\mepilex-border-flex-hqim005406.pdf
Creation Date (from metadata): D:20240422151454+02'00'

Processing URL: https://www.molnlycke.ca/contentassets/3a0ad2ec58d848169ae7391bb4370689/mepilexborderflex_productsheet_eng-1.pdf
Downloaded: downloads\mepilexborderflex_productsheet_eng-1.pdf
Creation Date (from metadata): D:20230327105115-04'00'

Processing URL: https://www.onemed.se/-/media/onemed/b2b/ligu/molnlycke/broschyr---mepilex-border-flex.pdf
Downloaded: downloads\broschyr---mepilex-border-flex.pdf
Creation Date (from metadata): D:20180219131323+01'00'

Processing URL: https://static.webareacontrol.com/CommonFile/mepilexborderflex-product-sheet-1647325121893.pdf
Downloaded: downloads\mepilexborderflex-product-sheet-1647325121893.pdf
Creation Date (from metadata): D:20171124115227-05'00'

Processing URL: https://www.molnlycke.lat/SysSite